# Exercise 7 - Local Trajectory Planning

In [ ]:
import numpy as np
from global_settings import get_params
from reference_path import ReferencePath
from trajectory import Trajectory, LateralCurve, LongitudinalCurve
from util import (
    transform_state_cartesian_to_frenet,
    transform_obstacles_frenet_to_cartesian,
    check_trajectories,
    ideal_tracking,
)
from frenet_vizualization import visualize
import operator

%matplotlib

### Step Function
Generate a set of trajectories and choose the best valid one.

In [ ]:
def step(state: dict, reference_path: ReferencePath, params: dict) -> tuple:
    """Generation of a trajectory

    :param state: current state of vehicle in frénet coordinates
    :type: dict
    :param reference_path: reference path
    :type: ReferencePath
    :param params: dictionary containing all parameters
    :type: dict
    :return: optimal trajectory, i.e. trajectory with lowest cost
    :rtype: Trajectory
    :return: dictionary containing all generated trajectories of this time step
    :rtype: dict"""

    # lists of curves and trajectories
    longitudinal_curves = []
    lateral_curves = []
    trajectories = []

    ###################################################################################
    ######################## Insert generation of trajectories ########################
    ###################################################################################

    # sort trajectories according to the cost associated with each trajectory
    trajectories = sorted(trajectories, key=operator.attrgetter("cost"))

    # check trajectories for validity and sort into valid and invalid
    trajectories = check_trajectories(trajectories=trajectories, parameters=params)

    # extract best trajectory (valid trajectory with lowest cost)
    optimal_trajectory = trajectories["valid"][0]

    return optimal_trajectory, trajectories

### Import parameters from params.ini and generate a ReferencePath object

In [ ]:
# get parameters from params.ini as dictionary
params = get_params()

# generate reference path
reference_path = ReferencePath(xy_waypoints=params["xy_waypoints"])

# calculate cartesian coordinates of obstacles and add to parameter dictionary
xy_obstacles = transform_obstacles_frenet_to_cartesian(
    reference_path=reference_path, sd_obstacles=params["sd_obstacles"]
)
params["xy_obstacles"] = xy_obstacles

### Set initial state in frénet coordinates

In [ ]:
# get initial cartesian state from parameter file
initial_cartesian_state = params["initial_state"]

# transform initial state from cartesian to frenét coordinate system
frenet_state = transform_state_cartesian_to_frenet(
    cartesian_state=initial_cartesian_state, reference_path=reference_path
)

### Main loop for Local Planning

In [ ]:
# number of steps
step_counter = 0

# dictionary to store all trajectories of every time step for the visualization
trajectories_over_time = dict()

# maximum progress s per step
max_s_progress_per_step = params["discretization"]["t_max"] * params["limits"]["v_max"]

# main loop
while frenet_state["s"] < reference_path.s_rp[-1] - max_s_progress_per_step:

    ####################################################################################
    ######################### Insert planning and control step #########################
    ####################################################################################

    # store trajectories of this time step in dictionary containing trajectories of all time steps for visualization
    trajectories_over_time[step_counter] = trajectories

    # print current progress every 20 steps
    if not step_counter % 20:
        progress = min(
            100
            * frenet_state["s"]
            / (reference_path.s_rp[-1] - max_s_progress_per_step),
            100,
        )
        print(f"Current progress {int(progress)} %")

    step_counter += 1

### Visualization of Local Planning

In [ ]:
# visualize trajectories of all steps
visualize(
    trajectories_over_time=trajectories_over_time,
    reference_path=reference_path,
    params=params,
)